In [ ]:
# Apply transfer learning with pre-trained VGG16/ResNet50MobileNet model on
# given dataset and analyze the results

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import VGG16, ResNet50, MobileNet
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_input_resnet
from tensorflow.keras.applications.mobilenet import preprocess_input as preprocess_input_mobilenet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

In [2]:
# Load CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the pixel values to [0, 1]
x_train, x_test = x_train / 255.0, x_test / 255.0

# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 14s 0us/step


In [4]:
# For VGG16
x_train_vgg16 = preprocess_input(x_train)
x_test_vgg16 = preprocess_input(x_test)

base_model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze the base model layers
for layer in base_model_vgg16.layers:
    layer.trainable = False

x = GlobalAveragePooling2D()(base_model_vgg16.output)
x = Dense(512, activation='relu')(x)
x = Dense(10, activation='softmax')(x)

model_vgg16 = Model(inputs=base_model_vgg16.input, outputs=x)
model_vgg16.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train VGG16
history_vgg16 = model_vgg16.fit(
    x_train_vgg16, y_train,
    epochs=1,
    validation_data=(x_test_vgg16, y_test),
    batch_size=64
)

# Evaluate VGG16
val_loss_vgg16, val_acc_vgg16 = model_vgg16.evaluate(x_test_vgg16, y_test)
print(f"VGG16 - Test Accuracy: {val_acc_vgg16:.4f}, Test Loss: {val_loss_vgg16:.4f}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 790s 1s/step - accuracy: 0.1252 - loss: 2.9641 - val_accuracy: 0.1684 - val_loss: 2.4202
313/313 ━━━━━━━━━━━━━━━━━━━━ 129s 411ms/step - accuracy: 0.1641 - loss: 2.4315
VGG16 - Test Accuracy: 0.1684, Test Loss: 2.4202


In [ ]:
# For ResNet50
x_train_resnet50 = preprocess_input_resnet(x_train)
x_test_resnet50 = preprocess_input_resnet(x_test)

base_model_resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze the base model layers
for layer in base_model_resnet50.layers:
    layer.trainable = False

x = GlobalAveragePooling2D()(base_model_resnet50.output)
x = Dense(512, activation='relu')(x)
x = Dense(10, activation='softmax')(x)

model_resnet50 = Model(inputs=base_model_resnet50.input, outputs=x)
model_resnet50.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# train
history_resnet50 = model_resnet50.fit(
    x_train_resnet50, y_train,
    epochs=5,
    validation_data=(x_test_resnet50, y_test),
    batch_size=64
)

val_loss_resnet50, val_acc_resnet50 = model_resnet50.evaluate(x_test_resnet50, y_test)
print(f"ResNet50 - Test Accuracy: {val_acc_resnet50:.4f}, Test Loss: {val_loss_resnet50:.4f}")

In [ ]:
# For MobileNet
x_train_mobilenet = preprocess_input_mobilenet(x_train)
x_test_mobilenet = preprocess_input_mobilenet(x_test)

base_model_mobilenet = MobileNet(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze the base model layers
for layer in base_model_mobilenet.layers:
    layer.trainable = False

x = GlobalAveragePooling2D()(base_model_mobilenet.output)
x = Dense(512, activation='relu')(x)
x = Dense(10, activation='softmax')(x)

model_mobilenet = Model(inputs=base_model_mobilenet.input, outputs=x)
model_mobilenet.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# train
history_mobilenet = model_mobilenet.fit(
    x_train_mobilenet, y_train,
    epochs=5,
    validation_data=(x_test_mobilenet, y_test),
    batch_size=64
)


# Evaluate MobileNet
val_loss_mobilenet, val_acc_mobilenet = model_mobilenet.evaluate(x_test_mobilenet, y_test)
print(f"MobileNet - Test Accuracy: {val_acc_mobilenet:.4f}, Test Loss: {val_loss_mobilenet:.4f}")

In [ ]:
# visualize
import matplotlib.pyplot as plt

def plot_history(history, model_name):
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label='val_accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title(f'{model_name} - Accuracy')
    plt.legend()
    plt.show()

    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title(f'{model_name} - Loss')
    plt.legend()
    plt.show()

# Plot results
plot_history(history_vgg16, 'VGG16')
plot_history(history_resnet50, 'ResNet50')
plot_history(history_mobilenet, 'MobileNet')
